In [5]:
source("functions/data.R")
source("functions/process.R")
require(neuralnet)

Loading required package: neuralnet


In [2]:
# Define dataframes
ETH <- get_data("ETH",params$start,params$end,params$period,52)
XRP <- get_data("XRP",params$start,params$end,params$period,52)
XMR <- get_data("XMR",params$start,params$end,params$period,52)
LTC <- get_data("LTC",params$start,params$end,params$period,52)

high <- cbind(ETH$high,XRP$high,XMR$high,LTC$high)
low <- cbind(ETH$low,XRP$low,XMR$low,LTC$low)
open <- cbind(ETH$open,XRP$open,XMR$open,LTC$open)
close <- cbind(ETH$close,XRP$close,XMR$close,LTC$close)

# Rename all the columns
colnames(high)  <- currency_vec
colnames(low) <- currency_vec
colnames(open) <- currency_vec
colnames(close) <- currency_vec
prices_vec <- list(high,low,open,close)

No encoding supplied: defaulting to UTF-8.


No encoding supplied: defaulting to UTF-8.


No encoding supplied: defaulting to UTF-8.


No encoding supplied: defaulting to UTF-8.


In [3]:
### Some more functions
# function that initializes weight vector given the number of assets (excluding cash)
init_weights <- function(n_assets){
    # element in first position (cash) is always initialized to 1
    return(c(1,rep(0,n_assets)))
}

# function that returns price tensor Xt given current vt and history n
# the tensor is returned as a feature vector where the columns of each matrix are listed in order
# i.e. first column of V_lo until last column of V_t
get_Xt <- function(vt,n){
    # initializes all three matrices
    V_lo <- getPriceMatrix(low,vt,n)
    V_hi <- getPriceMatrix(high,vt,n)
    V_t <- getPriceMatrix(close,vt,n)
    return(c(V_lo,V_hi,V_t))
}

# function that updates average returns (performance of policy) given counter and previous average
# this is done so that we don't need to store all returns
update_returns <- function(counter,returns,yt,wt){
    return((counter-1)/(counter)*returns + getLogReturns(yt,wt)/counter)
}

In [4]:
# simulation function, the agent randomizes the weight vector at each step
# params: n_hist, number of timesteps in the past for price history
# params: n_episodes, number of episodes in the simulation
simulate_random <- function(n_hist,n_episodes){
    prev_v <- tail(head(close,n_hist-1),1) ## initialize first previous price vector
    returns <- 0

    # simulation
    for(i in 0:n_episodes){

        # generates random action (portfolio vector)
        random_action <- function(n_assets){
        x <- runif(n_assets+1)
        return(x/sum(x))
        }

        # get price dataframes based on current time steps
        hi <- head(high,i+n_hist)
        lo <- head(low,i+n_hist)
        price <- head(close,i+n_hist)

        wt <- random_action(4)
        curr_v <- tail(price,1)
        yt <- getPriceRelativeVec(prev_v,curr_v)
        rt <- getLogReturns(yt,wt)

        print(paste0("episode ",i))
        print(cat("wt: ",wt))
        print(cat("yt: ",yt))
        print(cat("rt: ",rt))
        print("=================================")

        prev_v <- curr_v # set previous price as current price
        returns <- update_returns(i+1,returns,yt,wt)
    }

    print(paste("average returns for this simulation: ", returns))
}

simulate_random(10,100)

[1] "episode 0"
wt:  0.1955674 0.1679876 0.237115 0.001868758 0.3974612NULL
yt:  1 0.9725534 0.9732774 0.9997783 0.9930833NULL
rt:  -0.01379122NULL
[1] "================================="
[1] "episode 1"
wt:  0.209867 0.2609853 0.09768541 0.1363433 0.295119NULL
yt:  1 1.000992 0.9868102 0.9871119 0.9976934NULL
rt:  -0.003473442NULL
[1] "================================="
[1] "episode 2"
wt:  0.1764355 0.2988226 0.308072 0.01531389 0.201356NULL
yt:  1 1.001162 1.01773 1.025921 1.133885NULL
rt:  0.03262689NULL
[1] "================================="
[1] "episode 3"
wt:  0.1398533 0.03842881 0.3277352 0.2073841 0.2865987NULL
yt:  1 1.003188 1.007237 0.9957968 0.9659237NULL
rt:  -0.008176995NULL
[1] "================================="
[1] "episode 4"
wt:  0.283144 0.1756222 0.01834754 0.1502652 0.372621NULL
yt:  1 1.028732 1.032464 1.022991 1.040978NULL
rt:  0.02407365NULL
[1] "================================="
[1] "episode 5"
wt:  0.143622 0.2994037 0.03562992 0.4534809 0.06786349NULL
yt

In [14]:
# Try a new method based on sampling actions and approximating action value function

# function used to sample a random action
sample_action <- function(n_assets){
    x <- runif(n_assets+1)
    return(x/sum(x))
}

# Make Some Training Data
Var1 <- runif(50, 0, 100) 
# create a vector of 50 random values, min 0, max 100, uniformly distributed
sqrt.data <- data.frame(Var1, Sqrt=sqrt(Var1)) 
# create a dataframe with two columns, with Var1 as the first column
# and square root of Var1 as the second column

# Train the neural net
net.sqrt <- neuralnet(Sqrt~Var1,  sqrt.data, hidden=10, threshold=0.01)
# train a neural net, try and predict the Sqrt values based on Var1 values
# 10 hidden nodes

# Compute or predict for test data, (1:10)^2
compute(net.sqrt, (1:10)^2)$net.result
# What the above is doing is using the neural net trained (net.sqrt), 
# if we have a vector of 1^2, 2^2, 3^2 ... 10 ^2 (i.e. 1, 4, 9, 16, 25 ... 100), 
# what would net.sqrt produce?

1.254774474
1.997149954
2.994169568
4.001415595
4.997892565
6.002643974
6.999658074
7.998341659
9.003371897
9.972014956


In [43]:
# get price relative vector for starting state
price <- head(close,2)
yt <- getPriceRelativeVec(head(price,1),tail(price,1))
prev_v <- tail(price,1)
returns <- 0
data <- c()

for(i in 1:100){
    
    price <- head(close,i+2) # update price matrix 
    curr_v <- tail(price,1)
    yt_next <- getPriceRelativeVec(prev_v,curr_v) # get price relative vector of next state
    wt <- sample_action(4) # sample an action
    rt <- getLogReturns(yt_next,wt)
    
    data <- rbind(data,c(yt,wt,rt)) # append state action pair and reward to data
    
    prev_v <- curr_v
    yt <- yt_next
}

In [44]:
# create a testing dataframe
data <- data.frame(data)
data$X1 <- NULL
colnames(data) <- c("y2","y3","y4","y5","w1","w2","w3","w4","w5","r")

In [45]:
# fit neural net and predict
network <- neuralnet(r~y2+y3+y4+y5+w1+w2+w3+w4+w5,head(data,50),hidden=10,threshold=0.01)
cbind(compute(network,subset(tail(data,50),select=-r))$net.result,tail(data,50)$r)

51,-0.0079558480769,0.0708140701148
52,0.0245109623899,0.0672985527889
53,-0.0352382795099,-0.0049296967756
54,0.0147700499662,-0.0391479558467
55,0.0047419403723,0.0128816615937
56,0.0257578968284,-0.0069060369898
57,0.0173938037127,-0.0362066941949
58,0.0185186165525,-0.0327041260752
59,-0.0297483537749,0.0225316904661
60,0.0115562279471,-0.0168263977611
61,0.0055204617392,0.0022738971048
